In [5]:
import cv2
import os
import torch
from tqdm import tqdm
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from xml.etree import ElementTree as ET

# Define your bounding box color (e.g., red)
# Convert BGR to RGBA format (0-1 range)
bounding_box_color = (0, 0, 255)  # BGR format for red
bounding_box_color_rgb = (bounding_box_color[2] / 255.0, bounding_box_color[1] / 255.0, bounding_box_color[0] / 255.0, 1.0)

# Extend Visualizer to customize color handling
class MyVisualizer(Visualizer):
    def __init__(self, img_rgb, metadata, *args, **kwargs):
        super().__init__(img_rgb, metadata, *args, **kwargs)
    
    def draw_instance_predictions(self, predictions):
        for instance in predictions.pred_boxes:
            self.draw_box(instance.to("cpu"), edge_color=bounding_box_color_rgb)
        return self.output
    
# Load tire annotations
def load_tire_annotations(xml_path):
    tire_annotations = {}
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for track in root.findall('.//track'):
        label = track.attrib['label']
        if label == "tire":
            for box in track.findall('.//box'):
                frame_num = int(box.attrib['frame'])
                if frame_num not in tire_annotations:
                    tire_annotations[frame_num] = []
                tire_annotations[frame_num].append(box.attrib)
    return tire_annotations

# Configure the model
cfg = get_cfg()
config_file = "/home/isabelle.melo/proc/Mosquitoes/faster_detectron2/configs/faster_rcnn_R_50_FPN_1x/faster_rcnn_R_50_FPN_1x.yaml"
cfg.merge_from_file(config_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set the testing threshold for this model
cfg.MODEL.WEIGHTS = "outputs/faster_rcnn_R_50_FPN_1x/teste_augmentation/fold1_tire/model_early_stop.pth"  # path to the model we just trained
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Only one class
cfg.DATASETS.TRAIN = ('mbg_train1_tire',)
cfg.DATASETS.TEST = ("mbg_val1_tire", )

# Register dataset
MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes = ["tire"]

predictor = DefaultPredictor(cfg)

# Input and output directories
input_dir = '../dataset/v1/frames/video05/'
output_dir = 'output_frames/video05/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get the list of image files
image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

# Process each image in the input directory
for filename in tqdm(image_files, desc="Processing images"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)
    
    # Read the image
    image = cv2.imread(input_path)
    
    # Make predictions
    outputs = predictor(image)
    
    # Check if there are any predictions
    if len(outputs["instances"].pred_boxes) > 0:
        # Create custom visualizer
        v = MyVisualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2, instance_mode=ColorMode.SEGMENTATION)
        
        # Draw predictions on the image
        out_image = v.draw_instance_predictions(outputs["instances"]).get_image()[:, :, ::-1]
        
        # Save the output image
        cv2.imwrite(output_path, out_image)

print(f"Output images saved to {output_dir}")


Processing images: 100%|██████████| 42/42 [00:30<00:00,  1.37it/s]

Output images saved to output_frames/video05/


In [ ]:
import cv2
import os
import torch
from tqdm import tqdm
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from xml.etree import ElementTree as ET

# Define your bounding box color (e.g., red)
# Convert BGR to RGBA format (0-1 range)
bounding_box_color = (0, 0, 255)  # BGR format for red
bounding_box_color_rgb = (bounding_box_color[2] / 255.0, bounding_box_color[1] / 255.0, bounding_box_color[0] / 255.0, 1.0)

# Extend Visualizer to customize color handling
class MyVisualizer(Visualizer):
    def __init__(self, img_rgb, metadata, *args, **kwargs):
        super().__init__(img_rgb, metadata, *args, **kwargs)
    
    def draw_instance_predictions(self, predictions):
        for instance in predictions.pred_boxes:
            self.draw_box(instance.to("cpu"), edge_color=bounding_box_color_rgb)
        return self.output
    
# Load tire annotations
def load_tire_annotations(xml_path):
    tire_annotations = {}
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for track in root.findall('.//track'):
        label = track.attrib['label']
        if label == "tire":
            for box in track.findall('.//box'):
                frame_num = int(box.attrib['frame'])
                if frame_num not in tire_annotations:
                    tire_annotations[frame_num] = []
                tire_annotations[frame_num].append(box.attrib)
    return tire_annotations

# Configure the model
cfg = get_cfg()
config_file = "/home/isabelle.melo/proc/Mosquitoes/faster_detectron2/configs/faster_rcnn_R_50_FPN_1x/faster_rcnn_R_50_FPN_1x.yaml"
cfg.merge_from_file(config_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set the testing threshold for this model
cfg.MODEL.WEIGHTS = "outputs/faster_rcnn_R_50_FPN_1x/teste_augmentation/fold1_tire/model_early_stop.pth"  # path to the model we just trained
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Only one class
cfg.DATASETS.TRAIN = ('mbg_train1_tire',)
cfg.DATASETS.TEST = ("mbg_val1_tire", )

# Register dataset
MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes = ["tire"]

predictor = DefaultPredictor(cfg)

# Input and output directories
input_dir = '../dataset/v1/frames/video04/'
output_dir = 'output_frames/video04/'
annot_path = '../dataset/v1/annotations-xml/video04.xml'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get the list of image files
image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

# Process each image in the input directory
for filename in tqdm(image_files, desc="Processing images"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)
    
    # Read the image
    image = cv2.imread(input_path)
    
    # Make predictions
    outputs = predictor(image)
    
    tire_annotations = load_tire_annotations(annot_path)
    if filename in tire_annotations:
        for box_info in tire_annotations[filename]:
            xtl = int(float(box_info['xtl']))
            ytl = int(float(box_info['ytl']))
            xbr = int(float(box_info['xbr']))
            ybr = int(float(box_info['ybr']))
            thickness = 3
            cv2.rectangle(image, (xtl, ytl), (xbr, ybr), (0, 255, 0), thickness)
    
    # Check if there are any predictions
    if len(outputs["instances"].pred_boxes) > 0:
        # Create custom visualizer
        v = MyVisualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2, instance_mode=ColorMode.SEGMENTATION)
        
        # Draw predictions on the image
        out_image = v.draw_instance_predictions(outputs["instances"]).get_image()[:, :, ::-1]
        
        # Save the output image
        cv2.imwrite(output_path, out_image)

print(f"Output images saved to {output_dir}")

In [14]:
import cv2
import os
import torch
from tqdm import tqdm
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from xml.etree import ElementTree as ET
import re

# Define your bounding box color (e.g., red)
# Convert BGR to RGBA format (0-1 range)
bounding_box_color = (0, 0, 255)  # BGR format for red
bounding_box_color_rgb = (bounding_box_color[2] / 255.0, bounding_box_color[1] / 255.0, bounding_box_color[0] / 255.0, 1.0)

# Extend Visualizer to customize color handling
class MyVisualizer(Visualizer):
    def __init__(self, img_rgb, metadata, *args, **kwargs):
        super().__init__(img_rgb, metadata, *args, **kwargs)
    
    def draw_instance_predictions(self, predictions):
        for instance in predictions.pred_boxes:
            self.draw_box(instance.to("cpu"), edge_color=bounding_box_color_rgb)
        return self.output
    
# Load tire annotations
def load_tire_annotations(xml_path):
    tire_annotations = {}
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for track in root.findall('.//track'):
        label = track.attrib['label']
        if label == "tire":
            for box in track.findall('.//box'):
                frame_num = int(box.attrib['frame'])
                if frame_num not in tire_annotations:
                    tire_annotations[frame_num] = []
                tire_annotations[frame_num].append(box.attrib)
    return tire_annotations

# Configure the model
cfg = get_cfg()
config_file = "/home/isabelle.melo/proc/Mosquitoes/faster_detectron2/configs/faster_rcnn_R_50_FPN_1x/faster_rcnn_R_50_FPN_1x.yaml"
cfg.merge_from_file(config_file)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
cfg.MODEL.WEIGHTS = "outputs/faster_rcnn_R_50_FPN_1x/teste_augmentation/fold1_tire/model_early_stop.pth"  # path to the model we just trained
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Only one class
cfg.DATASETS.TRAIN = ('mbg_train1_tire',)
cfg.DATASETS.TEST = ("mbg_val1_tire", )

# Register dataset
MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes = ["tire"]

predictor = DefaultPredictor(cfg)

# Input and output directories
input_dir = '../dataset/v1/frames/video13/'
output_dir = 'output_frames/video13/'
annot_path = '../dataset/v1/annotations-xml/video13.xml'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get the list of image files
image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

# Regular expression pattern to match numbers in filenames
pattern = re.compile(r'frame_(\d+)\.png')

# Process each image in the input directory
for filename in tqdm(image_files, desc="Processing images"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)
    
    # Extract the frame number from the filename
    match = pattern.match(filename)
    frame_num = int(match.group(1)) if match else None
    
    # Read the image
    image = cv2.imread(input_path)
    
    # Make predictions
    outputs = predictor(image)
    
    tire_annotations = load_tire_annotations(annot_path)
    if frame_num in tire_annotations:
        print(f'frame_num: {frame_num}')
        for box_info in tire_annotations[frame_num]:
            xtl = int(float(box_info['xtl']))
            ytl = int(float(box_info['ytl']))
            xbr = int(float(box_info['xbr']))
            ybr = int(float(box_info['ybr']))
            thickness = 5
            cv2.rectangle(image, (xtl, ytl), (xbr, ybr), (255, 0, 0), thickness)
    
    # Check if there are any predictions
    if len(outputs["instances"].pred_boxes) > 0:
        # Create custom visualizer
        v = MyVisualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2, instance_mode=ColorMode.SEGMENTATION)
        
        # Draw predictions on the image
        out_image = v.draw_instance_predictions(outputs["instances"]).get_image()[:, :, ::-1]
        
        # Save the output image
        cv2.imwrite(output_path, out_image)

print(f"Output images saved to {output_dir}")


Processing images:   6%|▌         | 13/211 [00:05<01:16,  2.59it/s]

frame_num: 2760


Processing images:  32%|███▏      | 67/211 [00:24<00:50,  2.87it/s]

frame_num: 2952


Processing images:  38%|███▊      | 81/211 [00:29<00:43,  3.00it/s]

...

frame_num: 2856


Processing images:  95%|█████████▌| 201/211 [01:27<00:09,  1.02it/s]

frame_num: 2880


Processing images: 100%|██████████| 211/211 [01:31<00:00,  2.30it/s]

Output images saved to output_frames/video13/
